# Appendix: Verifying Claim 3.6.8

In [3]:
from itertools import combinations
from sage.matroids.advanced import *
from itertools import product

In [4]:
# The complete ternary Dowling geometry:
def DowlingGeometry(r):
    A = Matrix(GF(3), r, r*r)
    i = 0
    for j in range(r):
        A[j,i] = 1
        i += 1
    for j in range(r-1):
        for k in range(j+1,r):
            A[j,i] = -1
            A[k,i] = 1
            i += 1
            A[j,i] = -1
            A[k,i] = -1
            i += 1
    return Matroid(A)

In [5]:
def vname(path, ray, index):
    return path + str(ray) + '_' + str(index)
def ename(path, ray, index):
    return path + str(ray) + '_' + str(index)

In [6]:
def perturbed_gadget_matroid(perturb_matrix, gadgets_per_ray,
                             num_rays=1):
    """
    Create the matrix representing "rays" of gadgets joined by graph
    edges. In the case where perturb_matrix=[0 0 0 0], num_rays = 4,
    and gadgets_per_ray=[5,2,2,2], the result is M(J''') from
    Figure 5.

    Edges are encoded <path><ray>_<position>. So element 'p2_1' is the
    second edge on path 'p' from the third ray (counts start at 0).
    The other paths are q and r.

    gadgets_per_ray is a vector if num_rays > 1

    The gadget elements are labeled <label><ray>_<position>. The
    labels are d,e,f,g. The ``root'' gadget is `d0_0, e0_0, f0_0,
    g0_0'. In Figure 5, the root gadget is labeled F4.
    """
    gadget_block = Matrix(GF(3), [[0,1,0,1],
                                  [0,1,1,0],
                                  [0,0,1,1],
                                  [1,1,1,1]])  # matrix from Lemma 3.1
                                               # with the first six
                                               # columns removed
    if num_rays == 1:
        gadgets_per_ray = [gadgets_per_ray]
    num_rows = perturb_matrix.nrows() +
               sum(4 * g for g in gadgets_per_ray) -
               4 * (num_rays - 1) # Count root gadget only once.
    num_cols = sum(7 * (g - 1) for g in gadgets_per_ray) + 4
                # Each gadget other than the root gadget accounts for
                # seven elements -- the four elements of the gadget
                # and the three edges joining the gadget to the next
                # gadget. The root gadget gives four additional
                # elements.
    A = Matrix(GF(3), num_rows, num_cols)
    groundset = []
    vtx = {'p0_0': 0, 'q0_0': 1, 'r0_0': 2}  # Maps path vertices to
                                             # matrix rows
    imax = 3  # first unused row
    for ray in range(num_rays):
        vtx[vname('p',ray,0)] = vtx['p0_0']
        vtx[vname('q',ray,0)] = vtx['q0_0']
        vtx[vname('r',ray,0)] = vtx['r0_0']
        for i in range(1, gadgets_per_ray[ray]):
            vtx[vname('p',ray,i)] = imax
            vtx[vname('q',ray,i)] = imax + 1
            vtx[vname('r',ray,i)] = imax + 2
            imax += 3
    j = 0  # first unfilled column
    # Create the paths
    for ray in range(num_rays):
        for i in range(gadgets_per_ray[ray] - 1):
            # first path
            A[vtx[vname('p',ray,i)],j] = -1
            A[vtx[vname('p',ray,i+1)],j] = 1
            j += 1
            groundset.append(ename('p',ray,i))
            # second path
            A[vtx[vname('q',ray,i)],j] = -1
            A[vtx[vname('q',ray,i+1)],j] = 1
            j += 1
            groundset.append(ename('q',ray,i))
            # third path
            A[vtx[vname('r',ray,i)],j] = -1
            A[vtx[vname('r',ray,i+1)],j] = 1
            j += 1
            groundset.append(ename('r',ray,i))
    first_gadget = 0  # "root" gadget only gets added once
    for ray in range(num_rays):
        for k in range(first_gadget, gadgets_per_ray[ray]):
            first_gadget = 1 # "root" gadget only gets added once
            # glue on the gadgets
            elts = 'defg'
            for l in range(4):
                A[vtx[vname('p',ray,k)], j + l] = gadget_block[0, l]
                A[vtx[vname('q',ray,k)], j + l] = gadget_block[1, l]
                A[vtx[vname('r',ray,k)], j + l] = gadget_block[2, l]
                A[imax, j + l] = gadget_block[3, l]
            groundset.extend([ename(e,ray,k) for e in elts])
            imax += 1
            # put in the perturbation
            A.set_block(num_rows - perturb_matrix.nrows(), j,
                        perturb_matrix)
            j += 4
    return A, groundset

In [7]:
def check_perturbation(P):
    # Now we add a perturbation:
    A, gs = perturbed_gadget_matroid(P, [5,2,2,2], 4)
    M = Matroid(gs, A)
    contract_set = []
    # create negative loops on vertices of root gadget:
    contract_set.extend(['e1_1', 'f1_1', 'g1_1', 'e2_1', 'f2_1',
                         'g2_1', 'e3_1', 'f3_1', 'g3_1', 'p1_0',
                         'q2_0', 'r3_0'])
    # contract one element from each gadget on the main ray, except
    # the last:
    contract_set.extend(['d0_1', 'e0_2', 'f0_3', 'g0_4'])
    # contract the edges of the main ray:
    contract_set.extend([ename('p',0,i) for i in range(4)])
    contract_set.extend([ename('q',0,i) for i in range(4)])
    contract_set.extend([ename('r',0,i) for i in range(4)])
    N = (M / contract_set).simplify()
    return DowlingGeometry(N.rank()).has_minor(N) # We check if N is
                                                  # signed-graphic.

In [8]:
# We need to construct a list of all possible perturbations.
# We begin the list with the rank-0 perturbation.
perturbation_list=[Matrix(GF(3), [[0,0,0,0]])]

In [9]:
# Now we add all possible rank-1 perturbations.
bases_already_checked=[]
X=set([0,1,2,3])
for B in Subsets(X,1):
    nonB=X.difference(B)
    for T in product(range(3), repeat=3): # There are 3 entries
                                          # outside the identity
                                          # matrix.
        A=Matrix(GF(3), 1, 4)
        i=0
        for b in B:  # We make an identity matrix with columns
                     # labeled by the basis B.
            A[i,b]=1
            i=i+1
        A[0,list(nonB)[0]]=T[0]
        A[0,list(nonB)[1]]=T[1]
        A[0,list(nonB)[2]]=T[2]
        if all((A.matrix_from_columns(S)).det()==0 for S in
               bases_already_checked):
            # We only need to add a matrix to the list of
            # perturbations if it contains no basis for which we
            # already found all possible perturbations. This is
            # because row operations allow us to transform the
            # submatrix indexed by that basis into an identity matrix.
            perturbation_list.append(A)
    bases_already_checked.append(B)

In [10]:
# Now we add all possible rank-2 perturbations.
bases_already_checked=[]
X=set([0,1,2,3])
for B in Subsets(X,2):
    nonB=X.difference(B)
    for T in product(range(3), repeat=4): # There are 4 entries
                                          # outside the identity
                                          # matrix.
        A=Matrix(GF(3), 2, 4)
        i=0
        for b in B: # We make an identity matrix with columns
                    # labeled by the basis B.
            A[i,b]=1
            i=i+1
        A[0,list(nonB)[0]]=T[0]
        A[1,list(nonB)[0]]=T[1]
        A[0,list(nonB)[1]]=T[2]
        A[1,list(nonB)[1]]=T[3]
        if all((A.matrix_from_columns(S)).det()==0 for S in
               bases_already_checked):
            # We only need to add a matrix to the list of
            # perturbations if it contains no basis for which we
            # already found all possible perturbations. This is
            # because row operations allow us to transform the
            # submatrix indexed by that basis into an identity matrix.
            perturbation_list.append(A)
    bases_already_checked.append(B)

In [11]:
# Now we add all possible rank-3 perturbations.
bases_already_checked=[]
X=set([0,1,2,3])
for B in Subsets(X,3):
    nonB=X.difference(B)
    for T in product(range(3), repeat=3): # There are 3 entries
                                          # outside the identity
                                          # matrix.
        A=Matrix(GF(3), 3, 4)
        i=0
        for b in B: # We make an identity matrix with columns
                    # labeled by the basis B.
            A[i,b]=1
            i=i+1
        A[0,list(nonB)[0]]=T[0]
        A[1,list(nonB)[0]]=T[1]
        A[2,list(nonB)[0]]=T[2]
        if all((A.matrix_from_columns(S)).det()==0 for S in
               bases_already_checked):
            # We only need to add a matrix to the list of
            # perturbations if it contains no basis for which we
            # already found all possible perturbations. This is
            # because row operations allow us to transform the
            # submatrix indexed by that basis into an identity matrix.
            perturbation_list.append(A)
    bases_already_checked.append(B)

In [12]:
# Now we add the only possible rank-4 perturbation.
perturbation_list.append(matrix.identity(GF(3),4))

In [12]:
# We check all possible perturbations to see if any are
# signed-graphic matroids.
if any(check_perturbation(perturbation_list[t]) != False
       for t in range(len(perturbation_list))):
    print "signed-graphic matroid"
else:
    print "No perturbation is signed-graphic."

No perturbation is signed-graphic.


In [13]:
version()

'SageMath version 8.0, Release Date: 2017-07-21'